# Smart signatures
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Install PyTEAL
* Learn PyTEAL logic
* Write and deploy smart Signatures

### Install Pyteal

In [1]:
!pip install pyteal

## Setup
See notebook 04.1, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [2]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [3]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [4]:
from pyteal import *

In [5]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [6]:
from pyteal import *

In [7]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

ADLR27NB3V62INPRGCK63YMRWQCU4J2OE6PTIGZSZSXZNO5KSLAU7TPSFU
FEJNBD5DI3TC53AZNGSUAXAR6HAELQF4PLAHSVDUG3EP5WXEKRVULIWOJE
AH3NJBIMCZZLXY7NR2NENBWUS4P63JOJFUYM2O3DTW4J5E2ENQL5ISFGKU


#### Quick check of asset holdings, otherwise go to ...
- https://bank.testnet.algorand.network
- https://testnet.algoexplorer.io/dispenser

In [8]:
asset_holdings_df(algod_client,Alice['public'])

,amount,unit,asset-id,name,decimals
0,25.837,ALGO,0,Algorand,6
1,2.500,WSC,71140107,Peters WSC coin,2


#### Check Purestake API

In [18]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19795386


## Smart Signatures
* A Smart Signature is a function that has two possible results `True` or `False`
* A Smart Signature can only evaluate properties of a transaction that is proposed to it.
* A Smart Signature cannot read or write from/to the blockchain

## The Dispenser
* The simplest smart signature is always `TRUE`
* It accepts **all** transactions that are proposed

#### Step 1: The programmer writes down the conditions as a PyTeal program
* This is the logic of the smart signature. 
* Usually written inside `( ... )`
* This one is always `True`

In [27]:
import random
from random import randrange
a = Int( randrange(2**32-1) )   

dispenser_pyteal = (
    a == a
)

#### Step 2: Compile PyTeal -> Teal
* Necessary intermediate step
* No need to print and inspect the TEAL program

In [28]:
dispenser_teal = compileTeal(dispenser_pyteal, 
                             Mode.Signature,          # <----- Here we say it is a Smart Signature (and not a Smart Contract)
                             version=3)
print(dispenser_teal)

#pragma version 3
int 235811646
int 235811646
==


#### Step 3: Compile Teal -> Bytecode for AVM
AVM = Algorand Virtual Machine

`algod_client.compile` creates a dict with two entries:
* `hash` contains the address $\longleftarrow$ corresponds to the `[public]`
* `result` contains the compiled code $\longleftarrow$ corresponds to the `[private]`

In [29]:
Dispenser = algod_client.compile(dispenser_teal)
Dispenser

{'hash': 'FMJIAKUOKXQWCMRDY3FNLIWOQ5OX7SPEVJIOTEUMKTVYY4AOWTM5N77DVQ',
 'result': 'AyABvua4cCIiEg=='}

In [30]:
# Look on Algoexplorer at the address of the smart signature. 
# (There is not yet something to see)
print('http://testnet.algoexplorer.io/address/'+Dispenser['hash'])

http://testnet.algoexplorer.io/address/FMJIAKUOKXQWCMRDY3FNLIWOQ5OX7SPEVJIOTEUMKTVYY4AOWTM5N77DVQ


#### Step 4: Alice funds and deploys the smart signature
* Only *at this step* we decide who is funding the smart signature.
* This means we can write the Smart Signature without knowing who will fund it.
* It is even possible that multiple people fund a Smart Signature.
* Notice that the recipient is the **Dispenser**

In [31]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.1*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Dispenser['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19803525.
Waiting for round 19803525 to finish.
Waiting for round 19803526 to finish.
Transaction DNF4BOEPZFBQ5MNOXPHA2FXMOD2733OC5ONGKMSNMT7GRGCFHGMQ confirmed in round 19803527.


In [32]:
# Look at Algoexplorer. (The smart signature is funded.)
print('http://testnet.algoexplorer.io/address/'+Dispenser['hash'])

http://testnet.algoexplorer.io/address/FMJIAKUOKXQWCMRDY3FNLIWOQ5OX7SPEVJIOTEUMKTVYY4AOWTM5N77DVQ


#### Step 5: Alice informs Bob
*  Bob can only interact with the Smart signature, if he has the following information:

In [33]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Dispenser['result'])
print("Address of smart signature: ", Dispenser['hash'])

Alice communicates to Bob the following
Compiled smart signature: AyABvua4cCIiEg==
Address of smart signature:  FMJIAKUOKXQWCMRDY3FNLIWOQ5OX7SPEVJIOTEUMKTVYY4AOWTM5N77DVQ


#### Step 6: Bob proposes a transaction to the smart signature
* Using the information obtained in step 5
* He proposes a payment from the dispenser to himself
* The payment transaction is signed by the smart signature, **if the conditions are fullfilled** (easy in this case)

In [34]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0.2*1e6)
txn = PaymentTxn(sender=Dispenser['hash'], sp=sp, receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Dispenser['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19803984.
Waiting for round 19803984 to finish.
Waiting for round 19803985 to finish.
Transaction 4MVVZ4FWFTERI7FY45INOHZ7GISRRSZOJ7GYTI4SRIS4HPSNFNIA confirmed in round 19803986.


In [35]:
# Look again at Algoexplorer. 
# - The smart signature has fewer ALGOs.
# - Bob has more ALGOs.
print('http://testnet.algoexplorer.io/address/'+Dispenser['hash'])
print('http://testnet.algoexplorer.io/address/'+Bob['public'])

http://testnet.algoexplorer.io/address/FMJIAKUOKXQWCMRDY3FNLIWOQ5OX7SPEVJIOTEUMKTVYY4AOWTM5N77DVQ
http://testnet.algoexplorer.io/address/FEJNBD5DI3TC53AZNGSUAXAR6HAELQF4PLAHSVDUG3EP5WXEKRVULIWOJE


##### Check holdings

### Exercise
* Run step 6 again and check again holdings

### Exercise
* Charlie wants to get **all** the money in the Smart Signature.
* How much can he withdraw?
* Do not forget the ...

In [28]:
# Hint: this is the amount of micro-Algos currently in the Smart Signaure
algod_client.account_info(Dispenser['hash'])['amount']

17798000

In [ ]:
# Python code goes here